# CI/CD 자동화 실습: GitHub Actions & Azure Pipelines
이 노트북에서는 GitHub Actions와 Azure Pipelines를 활용하여 컨테이너 기반 애플리케이션의 CI/CD 자동화 실습을 진행합니다.

## 1. CI/CD란?
- **CI(Continuous Integration)**: 코드 변경 사항을 자동으로 빌드 및 테스트
- **CD(Continuous Delivery/Deployment)**: 애플리케이션을 자동으로 배포

### CI/CD의 이점
- 빠른 피드백
- 자동화된 품질 보증
- 신속한 배포

## 2. GitHub Actions로 CI/CD 구현
GitHub Actions는 GitHub 저장소에서 직접 워크플로우를 자동화할 수 있는 도구입니다.

### 예시 워크플로우 (build-and-push.yml)
아래는 컨테이너 이미지를 빌드하고 ACR로 푸시하는 GitHub Actions 워크플로우 예시입니다.

In [ ]:
# .github/workflows/build-and-push.yml
name: Build and Push to ACR
on:
  push:
    branches: [ main ]
jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      - name: Azure Login
        uses: azure/login@v2
        with:
          creds: ${{ secrets.AZURE_CREDENTIALS }}
      - name: Build and Push
        uses: azure/docker-login@v1
        with:
          login-server: myacr.azurecr.io
          username: ${{ secrets.ACR_USERNAME }}
          password: ${{ secrets.ACR_PASSWORD }}
      - run: |
          docker build -t myacr.azurecr.io/myflaskapp:latest .
          docker push myacr.azurecr.io/myflaskapp:latest

## 3. Azure Pipelines로 CI/CD 구현
Azure Pipelines는 다양한 플랫폼에서 빌드, 테스트, 배포를 자동화할 수 있는 Azure DevOps의 서비스입니다.

### 예시 파이프라인 (azure-pipelines.yml)
아래는 컨테이너 이미지를 빌드하고 ACR로 푸시하는 Azure Pipelines 예시입니다.

In [ ]:
# azure-pipelines.yml
trigger:
- main

pool:
  vmImage: 'ubuntu-latest'

steps:
- checkout: self
- task: AzureCLI@2
  inputs:
    azureSubscription: 'AzureServiceConnection'
    scriptType: 'bash'
    scriptLocation: 'inlineScript'
    inlineScript: |
      az acr login --name myacr
      docker build -t myacr.azurecr.io/myflaskapp:latest .
      docker push myacr.azurecr.io/myflaskapp:latest

## 4. ArgoCD를 활용한 GitOps 방식 배포
ArgoCD는 쿠버네티스 환경에서 GitOps 방식을 실현하는 대표적인 오픈소스 도구입니다. Git 저장소의 선언적 매니페스트를 기준으로 애플리케이션을 자동으로 배포 및 동기화합니다.

### GitOps란?
- 모든 배포/운영 구성을 Git 저장소에 선언적으로 관리
- 변경 사항이 감지되면 자동으로 쿠버네티스 클러스터에 적용

### ArgoCD 설치 (AKS 클러스터)
```bash
kubectl create namespace argocd
kubectl apply -n argocd -f https://raw.githubusercontent.com/argoproj/argo-cd/stable/manifests/install.yaml
```

### ArgoCD CLI 설치 (로컬)
macOS 기준:
```bash
brew install argocd
```

### ArgoCD 서버 접속 및 초기 로그인
```bash
kubectl port-forward svc/argocd-server -n argocd 8080:443
# 브라우저에서 https://localhost:8080 접속
# 초기 admin 패스워드 확인
kubectl get secret argocd-initial-admin-secret -n argocd -o jsonpath="{.data.password}" | base64 -d
```

### 애플리케이션 등록 및 배포
아래는 예시 manifest 저장소를 등록하고 배포하는 방법입니다.
```bash
argocd login localhost:8080
argocd app create myapp \
  --repo https://github.com/your-org/your-repo.git \
  --path k8s-manifests \
  --dest-server https://kubernetes.default.svc \
  --dest-namespace default
argocd app sync myapp
```

### 참고
- 실제 배포 환경에서는 인증, RBAC, 외부 도메인 설정 등 추가 구성이 필요할 수 있습니다.
- 자세한 내용은 [ArgoCD 공식 문서](https://argo-cd.readthedocs.io/)를 참고하세요.

## 5. 정리

이번 실습에서는 GitHub Actions, Azure Pipelines, ArgoCD를 활용해 컨테이너 이미지 빌드부터 ACR 푸시, AKS 배포, GitOps 기반 선언적 배포까지 CI/CD 자동화 전 과정을 실습했습니다. 각 도구의 자동화 파이프라인을 통해 효율적이고 일관된 배포 환경을 구축할 수 있습니다.

이제 컨테이너 이미지를 자동으로 빌드하고, ACR로 푸시하며, AKS에 배포하는 전체 자동화 파이프라인과 GitOps 기반 배포까지 구축할 수 있습니다.